# PDF to excel conversion code



In [1]:
#Importing all the required liabraries

import re

import pdfplumber
import pandas as pd
from collections import namedtuple

In [2]:
#Creating a tuple consists of the column wise data bifurcation

Line = namedtuple('Line', 'company_id company_name doctype reference currency voucher inv_date due_date open_amt_tc open_amt_bc current months1 months2 months3')

In [3]:
#Creating company and line reg expressions

company_re = re.compile(r'(V\d+) (.*) Phone:')
line_re = re.compile(r'\d{2}/\d{2}/\d{4} \d{2}/\d{2}/\d{4}')

In [4]:
#Checking the grouping of the characters to their reg expressions categories

company_re.search('V50012 Company A LTD Phone:2123232323').group(2)

'Company A LTD'

In [5]:
#Splitting the column wise data to their respective fields and formats

Line(*'company_no company_name INVOICE 478 USD BFS030OB/000000057 11/03/2014 12/30/2014 263.52 263.52 0.00 0.00 0.00 263.52'.split())

Line(company_id='company_no', company_name='company_name', doctype='INVOICE', reference='478', currency='USD', voucher='BFS030OB/000000057', inv_date='11/03/2014', due_date='12/30/2014', open_amt_tc='263.52', open_amt_bc='263.52', current='0.00', months1='0.00', months2='0.00', months3='263.52')

In [6]:
#Defining pdf file to convert

file = 'Sample Report Pythonic.pdf'

In [7]:
# pdf to excel conversion code by splitting basis the format of each element specified in the pdf

lines = []
total_check = 0

with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text = page.extract_text()
        for line in text.split('\n'):
            comp = company_re.search(line)
            if comp:
                vend_no, vend_name = comp.group(1),comp.group(2)
                
            elif line.startswith('INVOICES'):
                doctype = 'INVOICE'
                
            elif  line.startswith('CREDITNOTES'):
                doctype = 'CREDITNOTE'
                
            elif line_re.search(line):
                items = line.split()
                lines.append(Line(vend_no, vend_name, doctype, *items))
                
            elif line.startswith('Supplier total'):
                tot = float(line.split()[2].replace(',', ''))
                total_check += tot

In [8]:
lines[:5]

[Line(company_id='V50012', company_name='Company A LTD', doctype='CREDITNOTE', reference='478', currency='USD', voucher='BFS030OB/000000057', inv_date='11/03/2014', due_date='12/30/2014', open_amt_tc='263.52', open_amt_bc='263.52', current='0.00', months1='0.00', months2='0.00', months3='263.52'),
 Line(company_id='V50012', company_name='CARGO SAFETRAVEL LTD', doctype='INVOICE', reference='J1062628/1', currency='USD', voucher='BFS026/000001137', inv_date='03/30/2019', due_date='03/30/2019', open_amt_tc='-102.00', open_amt_bc='-102.00', current='0.00', months1='-102.00', months2='0.00', months3='0.00'),
 Line(company_id='V50012', company_name='CARGO SAFETRAVEL LTD', doctype='INVOICE', reference='J1062638/1', currency='USD', voucher='BFS026/000001136', inv_date='02/28/2019', due_date='03/15/2019', open_amt_tc='-204.00', open_amt_bc='-204.00', current='0.00', months1='-204.00', months2='0.00', months3='0.00'),
 Line(company_id='V50014', company_name='ChaseB REGISTER USDA', doctype='INVOIC

In [9]:
# Creating the dataframe of the extracted data and viewing the top 5 records
df = pd.DataFrame(lines)
df.head()

,company_id,company_name,doctype,reference,currency,voucher,inv_date,due_date,open_amt_tc,open_amt_bc,current,months1,months2,months3
0,V50012,Company A LTD,CREDITNOTE,478,USD,BFS030OB/000000057,11/03/2014,12/30/2014,263.52,263.52,0.00,0.00,0.00,263.52
1,V50012,CARGO SAFETRAVEL LTD,INVOICE,J1062628/1,USD,BFS026/000001137,03/30/2019,03/30/2019,-102.00,-102.00,0.00,-102.00,0.00,0.00
2,V50012,CARGO SAFETRAVEL LTD,INVOICE,J1062638/1,USD,BFS026/000001136,02/28/2019,03/15/2019,-204.00,-204.00,0.00,-204.00,0.00,0.00
3,V50014,ChaseB REGISTER USDA,INVOICE,110-135633,USD,BFS026/000001064,02/05/2019,04/29/2019,"-4,665.00","-4,665.00","-4,665.00",0.00,0.00,0.00
4,V50015,METALS CHEMICALS GROUP LIMITED,INVOICE,SI191433,USD,BFS026/000000908,02/12/2019,03/30/2019,-198.00,-198.00,0.00,-198.00,0.00,0.00


In [10]:
#Checking the datatype

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_id    131 non-null    object
 1   company_name  131 non-null    object
 2   doctype       131 non-null    object
 3   reference     131 non-null    object
 4   currency      131 non-null    object
 5   voucher       131 non-null    object
 6   inv_date      131 non-null    object
 7   due_date      131 non-null    object
 8   open_amt_tc   131 non-null    object
 9   open_amt_bc   131 non-null    object
 10  current       131 non-null    object
 11  months1       131 non-null    object
 12  months2       131 non-null    object
 13  months3       131 non-null    object
dtypes: object(14)
memory usage: 14.5+ KB


### Data Observations
1. Inv_date and due_date format is object wherein it should be convert to datetime format
2. Amount columns also need to change from object to float regardless of the currency in which it has recorded in pdf

In [11]:
#Changing the date format from object to datetime

df['inv_date'] = pd.to_datetime(df['inv_date'])
df['due_date'] = pd.to_datetime(df['due_date'])

In [12]:
#  Need to convert the amount column type to float
for col in df.columns[-6:]:
    df[col] = df[col].map(lambda x: float(str(x).replace(',', '')))



In [13]:
#Lets recheck the format for whole data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   company_id    131 non-null    object        
 1   company_name  131 non-null    object        
 2   doctype       131 non-null    object        
 3   reference     131 non-null    object        
 4   currency      131 non-null    object        
 5   voucher       131 non-null    object        
 6   inv_date      131 non-null    datetime64[ns]
 7   due_date      131 non-null    datetime64[ns]
 8   open_amt_tc   131 non-null    float64       
 9   open_amt_bc   131 non-null    float64       
 10  current       131 non-null    float64       
 11  months1       131 non-null    float64       
 12  months2       131 non-null    float64       
 13  months3       131 non-null    float64       
dtypes: datetime64[ns](2), float64(6), object(6)
memory usage: 14.5+ KB


In [14]:
df['open_amt_bc'].sum()

-129580.73999999999

In [15]:
total_check

-129580.73999999999

In [16]:
# Saving the formatted file in .csv format
df.to_csv('Invoices.csv', index=False)  #csv format